In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher

In [2]:
# function that filter the urls and symbols in the text 
def clean_text(df_text):
    def filter_text(x):
        url = 'http[s]?://\S+'
        x = re.sub(url,'',x)
        x = re.sub("[^\w\s]",' ',x) # filter symbols
        x = re.sub("\s+",' ',x)
        ls=[w.lower() for w in x.split()] 
        return ' '.join(ls)
    df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
    df_text['text'] = df_text['text'].replace('/u0026', ' ')
    df_text.drop_duplicates(subset = 'text',keep = 'first', inplace = True)
    df_text.reset_index(drop = True,inplace = True)
    return df_text

In [3]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    nlp = spacy.load('en_core_web_lg')
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        matcher = PhraseMatcher(nlp.vocab)
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", patterns) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')

In [4]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)

In [5]:
def get_word_lable(df_text,lexicon):
    with open (lexicon) as f:
        dic = json.load(f)
    df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic))
    df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic))
    df_text['m_label'] = df_text['m_label'].apply(lambda x: 'no topic' if x=='' else x)
    return df_text

In [6]:
def count_topic(df_text):
    df_tag = df_text['m_label'].value_counts().rename_axis('topics').reset_index(name='counts')
    df_tag = df_tag.assign(single_topic=df_tag['topics'].str.split(',')).explode('single_topic').reset_index(drop = True)
    df_tag = df_tag.groupby('single_topic').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_tag.reset_index(drop = True, inplace = True)
    return df_tag

In [7]:
def count_word(df_text):
    df_words = df_text['words'].value_counts().rename_axis('words').reset_index(name='counts')
    df_words = df_words.assign(single_word=df_words['words'].str.split(',')).explode('single_word').reset_index(drop = True)
    df_words = df_words.groupby('single_word').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_words.reset_index(drop = True, inplace = True)
    return df_words

In [12]:
# go through the csv file, get youtube video's ids from all video ads. Add new column 'youtube_id' 
# before running this function, make sure the csv file includes ad_url
def get_youtube_id(fn): 
    df = pd.read_csv(fn)
    df = df[df['ad_type'] == 'Video'].reset_index(drop = True) # only get video ads
    df_video = df[['ad_url','advertiser_name','impressions','spend_usd']].reset_index(drop = True) 
    urls = df_video['ad_url'].to_list() # 
    youtube_ids = [] # get youtube id
    for url in urls:
        entity_id = url.split('/')[-3]
        creative_id = url.split('/')[-1]
        report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
        response = requests.get(report_url)
        try:
            youtube_id = response.text.split('"')[3]
            if len(list(youtube_id)) > 11: # mark ads violating google polices
                youtube_id = 'youtube_id not available: this ad violated google ad policies.' 
            #print(youtube_id + ', ' + creative_id)
        except IndexError: # mark ads cannot be loaded
            youtube_id = 'youtube_id not available: cannot load the video with this ad_url.'
            #print("can't load this video, " + report_url)
            pass
        youtube_ids.append(youtube_id)
    df_video['youtube_id'] = youtube_ids
    return df_video

In [14]:
# check all videos, see which are available and drop duplicates. Add a new column video_available
def check_video(df_video): 
  for id in df_video['youtube_id']:
    if len(list(id)) == 11:
      df_video.drop_duplicates(subset = 'youtube_id',keep = 'first', inplace = True)
      df_video.reset_index(drop = True, inplace=True)
      yes_video = df_video['youtube_id'].str.len() == 11
      df_video['video_available'] = yes_video
      return df_video

In [13]:
# use youtube_id to get captions. Add a new column youtube_captions
def get_captions(df_video):
  youtube_captions = []
  for youtube_id in df_video['youtube_id']:
    try:
      subs = YouTubeTranscriptApi.get_transcript(youtube_id)
      alist = []
      for sub in subs:
        alist.append(" " + sub['text'])
      captions = ""
      for item in alist:
        captions += item
    except Exception as e:
      captions = e   
    youtube_captions.append(captions)
  #print(len(youtube_captions))
  df_video['text'] = youtube_captions
  return df_video

# CA

In [15]:
ca_video = get_youtube_id('GoogleAds/CA.csv')
ca_video_clean = check_video(ca_video)
ca_captions = get_captions(ca_video_clean)
ca_captions

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders sum...
1,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,C0Bwn5e2Fqc,True,[Music] [Music] all right hello and welcome a...
2,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,O8v9Nk_PKpM,True,i am a big uh supporter of the highway 37 exp...
3,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,youtube_id not available: cannot load the vide...,False,\nCould not retrieve a transcript for the vide...
4,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,phLp03VtiSg,True,hello and welcome everyone to another episode...
...,...,...,...,...,...,...,...
62,https://transparencyreport.google.com/politica...,RESOURCE MEDIA A NONPROFIT CORPORATION,≤ 10k,≤ 100,ZFsQpVfsBro,True,"My life before, was like a train wreck, is th..."
63,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,vfQVWp6Bw8A,True,[Music] welcome to the lost debate on convent...
64,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,EPrj6MEu_EU,True,[Music] welcome to the lost debate a show for...
65,https://transparencyreport.google.com/politica...,KATIE PORTER FOR CONGRESS,10k-100k,100-1k,L5y4VxGyQ2s,True,hi it's representative katie porter i wanted ...


In [16]:
# remove unavialable videos
ca_captions = ca_captions[ca_captions['video_available'] == True]
ca_captions_clean = clean_text(ca_captions)
ca_captions_clean

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:12: SettingWithCopyWarning: 
A value is try

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...
1,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,C0Bwn5e2Fqc,True,music music all right hello and welcome again ...
2,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,O8v9Nk_PKpM,True,i am a big uh supporter of the highway 37 expa...
3,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,phLp03VtiSg,True,hello and welcome everyone to another episode ...
4,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,IUQYgYSl0Pc,True,yeah kind of as as we re kind of coming to a c...
...,...,...,...,...,...,...,...
57,https://transparencyreport.google.com/politica...,RESOURCE MEDIA A NONPROFIT CORPORATION,≤ 10k,≤ 100,ZFsQpVfsBro,True,my life before was like a train wreck is the b...
58,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,vfQVWp6Bw8A,True,music welcome to the lost debate on convention...
59,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,EPrj6MEu_EU,True,music welcome to the lost debate a show for po...
60,https://transparencyreport.google.com/politica...,KATIE PORTER FOR CONGRESS,10k-100k,100-1k,L5y4VxGyQ2s,True,hi it s representative katie porter i wanted t...


In [17]:
ca_captions_tagged = get_word_lable(ca_captions_clean,'lexicon.json')
ca_captions_tagged

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:6: SettingWithCopyWarn

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text,words,m_label
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...,"greenhouse gas,workers,climate,pollution,trans...","economic,environment,health"
1,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,C0Bwn5e2Fqc,True,music music all right hello and welcome again ...,"healthcare,economic,health care,teachers,crimi...","covid,donation,economic,education,environment,..."
2,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,O8v9Nk_PKpM,True,i am a big uh supporter of the highway 37 expa...,"climate,sea level",environment
3,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,phLp03VtiSg,True,hello and welcome everyone to another episode ...,"laws,marijuana,guns,landlord,safety,gun,racial...","covid,economic,education,governance,health,imm..."
4,https://transparencyreport.google.com/politica...,JARHETT BLONIEN,≤ 10k,≤ 100,IUQYgYSl0Pc,True,yeah kind of as as we re kind of coming to a c...,"tax,deficit,tribes,surpluses","economic,social and cultural"
...,...,...,...,...,...,...,...,...,...
57,https://transparencyreport.google.com/politica...,RESOURCE MEDIA A NONPROFIT CORPORATION,≤ 10k,≤ 100,ZFsQpVfsBro,True,my life before was like a train wreck is the b...,"heroin,incarcerated,black communities,environm...","economic,education,environment,governance,heal..."
58,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,vfQVWp6Bw8A,True,music welcome to the lost debate on convention...,"vaccine mandates,laws,foreign,house floor,disc...","covid,economic,education,environment,foreign p..."
59,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,EPrj6MEu_EU,True,music welcome to the lost debate a show for po...,"laws,discrimination,distancing,impeach,prison,...","covid,economic,education,environment,foreign p..."
60,https://transparencyreport.google.com/politica...,KATIE PORTER FOR CONGRESS,10k-100k,100-1k,L5y4VxGyQ2s,True,hi it s representative katie porter i wanted t...,no words,no topic


In [18]:
ca_topic = count_topic(ca_captions_tagged)
ca_topic

,single_topic,counts
0,economic,39
1,safety,31
2,governance,30
3,social and cultural,29
4,health,28
5,covid,27
6,education,27
7,environment,24
8,social programs,24
9,foreign policy,23


In [19]:
ca_word = count_word(ca_captions_tagged)
ca_word

,single_word,counts
0,health,23
1,pandemic,22
2,school,22
3,law,20
4,education,17
...,...,...
500,gold star families,1
501,give tax,1
502,ghani,1
503,genocides,1


# TX

In [25]:
tx_video = get_youtube_id('GoogleAds/texas.csv')
tx_video_clean = check_video(tx_video)
tx_captions = get_captions(tx_video_clean)
tx_captions

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders sum...
1,https://transparencyreport.google.com/politica...,Chad Crow,≤ 10k,≤ 100,7ne91Qj4XEo,True,i believe that free and fair elections are on...
2,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,youtube_id not available: cannot load the vide...,False,\nCould not retrieve a transcript for the vide...
3,https://transparencyreport.google.com/politica...,Allen West,≤ 10k,≤ 100,9i4lMGfNTRA,True,hey i'm ted nugent full-time i'm addicted to ...
4,https://transparencyreport.google.com/politica...,JESSICA CISNEROS FOR CONGRESS,≤ 10k,≤ 100,B1ozJ2JH8BE,True,tonight fbi agents at the home of henry cuell...
...,...,...,...,...,...,...,...
150,https://transparencyreport.google.com/politica...,Hohman for Congress,10k-100k,100-1k,8Jf4sxrOkzo,True,you know the drone is overhead but you can't ...
151,https://transparencyreport.google.com/politica...,John N. Raney,10k-100k,100-1k,_Ff2-uWTpl0,True,texans can count on john rainey rainey is fig...
152,https://transparencyreport.google.com/politica...,TEXANS FOR HENRY CUELLAR CONGRESSIONAL CAMPAIGN,10k-100k,100-1k,idEsiU0jBiA,True,[Music] get up early work hard my parents sai...
153,https://transparencyreport.google.com/politica...,Doctor Matt Rostami For Texas Campaign,10k-100k,100-1k,4ynkJPbh4Ns,True,to grow the republican party we're going to n...


In [28]:
# remove unavialable videos and clean texts
tx_captions_available = tx_captions[tx_captions['video_available'] == True]
tx_captions_clean = clean_text(tx_captions_available)
tx_captions_clean

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:12: SettingWithCopyWarning: 
A value is try

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...
1,https://transparencyreport.google.com/politica...,Chad Crow,≤ 10k,≤ 100,7ne91Qj4XEo,True,i believe that free and fair elections are one...
2,https://transparencyreport.google.com/politica...,Allen West,≤ 10k,≤ 100,9i4lMGfNTRA,True,hey i m ted nugent full time i m addicted to f...
3,https://transparencyreport.google.com/politica...,JESSICA CISNEROS FOR CONGRESS,≤ 10k,≤ 100,B1ozJ2JH8BE,True,tonight fbi agents at the home of henry cuella...
4,https://transparencyreport.google.com/politica...,JESSICA CISNEROS FOR CONGRESS,≤ 10k,≤ 100,noinAynqn28,True,could not retrieve a transcript for the video ...
...,...,...,...,...,...,...,...
135,https://transparencyreport.google.com/politica...,Hohman for Congress,10k-100k,100-1k,8Jf4sxrOkzo,True,you know the drone is overhead but you can t h...
136,https://transparencyreport.google.com/politica...,John N. Raney,10k-100k,100-1k,_Ff2-uWTpl0,True,texans can count on john rainey rainey is figh...
137,https://transparencyreport.google.com/politica...,TEXANS FOR HENRY CUELLAR CONGRESSIONAL CAMPAIGN,10k-100k,100-1k,idEsiU0jBiA,True,music get up early work hard my parents said t...
138,https://transparencyreport.google.com/politica...,Doctor Matt Rostami For Texas Campaign,10k-100k,100-1k,4ynkJPbh4Ns,True,to grow the republican party we re going to ne...


In [29]:
tx_captions_tagged = get_word_lable(tx_captions_clean,'lexicon.json')
tx_captions_tagged

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:6: SettingWithCopyWarn

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text,words,m_label
0,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...,"greenhouse gas,workers,climate,pollution,trans...","economic,environment,health"
1,https://transparencyreport.google.com/politica...,Chad Crow,≤ 10k,≤ 100,7ne91Qj4XEo,True,i believe that free and fair elections are one...,"election integrity,voter fraud",governance
2,https://transparencyreport.google.com/politica...,Allen West,≤ 10k,≤ 100,9i4lMGfNTRA,True,hey i m ted nugent full time i m addicted to f...,"addicted,constitution","governance,safety"
3,https://transparencyreport.google.com/politica...,JESSICA CISNEROS FOR CONGRESS,≤ 10k,≤ 100,B1ozJ2JH8BE,True,tonight fbi agents at the home of henry cuella...,"corruption,fbi agents","governance,safety"
4,https://transparencyreport.google.com/politica...,JESSICA CISNEROS FOR CONGRESS,≤ 10k,≤ 100,noinAynqn28,True,could not retrieve a transcript for the video ...,"turkish,khmer,estonian,kurdish,persian",foreign policy
...,...,...,...,...,...,...,...,...,...
135,https://transparencyreport.google.com/politica...,Hohman for Congress,10k-100k,100-1k,8Jf4sxrOkzo,True,you know the drone is overhead but you can t h...,"vaccine mandates,big government,secure the bor...","covid,foreign policy,governance,immigration,sa..."
136,https://transparencyreport.google.com/politica...,John N. Raney,10k-100k,100-1k,_Ff2-uWTpl0,True,texans can count on john rainey rainey is figh...,"police,law,secure the border,border","governance,immigration,safety"
137,https://transparencyreport.google.com/politica...,TEXANS FOR HENRY CUELLAR CONGRESSIONAL CAMPAIGN,10k-100k,100-1k,idEsiU0jBiA,True,music get up early work hard my parents said t...,"border security,middle class,health care,tax,j...","economic,governance,health,immigration"
138,https://transparencyreport.google.com/politica...,Doctor Matt Rostami For Texas Campaign,10k-100k,100-1k,4ynkJPbh4Ns,True,to grow the republican party we re going to ne...,"expand medicaid,insurance companies,health car...","health,social programs"


In [30]:
count_topic(tx_captions_tagged)

,single_topic,counts
0,governance,60
1,economic,48
2,immigration,38
3,safety,34
4,social and cultural,33
5,foreign policy,30
6,health,27
7,social programs,23
8,covid,22
9,education,22


In [31]:
count_word(tx_captions_tagged)

,single_word,counts
0,law,39
1,border,32
2,police,18
3,economy,17
4,health,15
...,...,...
461,nominees,1
462,north korea,1
463,nra,1
464,nsa,1


# NY

In [32]:
ny_video = get_youtube_id('GoogleAds/NY.csv')
ny_video_clean = check_video(ny_video)
ny_captions = get_captions(ny_video_clean)
ny_captions

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there's no...
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your natio...
2,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,[Music] welcome to the lost debate a show for...
3,https://transparencyreport.google.com/politica...,ZELDIN FOR NEW YORK,≤ 10k,≤ 100,youtube_id not available: this ad violated goo...,False,\nCould not retrieve a transcript for the vide...
4,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,UcQHDSvtl90,True,[Music] my dad started at a steel plant and t...
5,https://transparencyreport.google.com/politica...,MARKETFUEL SUBSCRIPTION SERVICES,≤ 10k,≤ 100,jG6NGdLUHeM,True,general michael flynn was unmasked at least 4...
6,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,GwiWAmH9Pd4,True,i'm fighting to help families keep up with ri...
7,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah he...
8,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,zyu4BUYbf4U,True,38 states have officially approved the bounda...
9,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,GsX_SUh-p7U,True,with all of the news about the pandemic and t...


In [33]:
# remove unavialable videos
ny_captions_available = ny_captions[ny_captions['video_available'] == True]
ny_captions_clean = clean_text(ny_captions_available)
ny_captions_clean

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:12: SettingWithCopyWarning: 
A value is try

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there s no ...
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your nation...
2,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,music welcome to the lost debate a show for po...
3,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,UcQHDSvtl90,True,music my dad started at a steel plant and taug...
4,https://transparencyreport.google.com/politica...,MARKETFUEL SUBSCRIPTION SERVICES,≤ 10k,≤ 100,jG6NGdLUHeM,True,general michael flynn was unmasked at least 48...
5,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,GwiWAmH9Pd4,True,i m fighting to help families keep up with ris...
6,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah her...
7,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,zyu4BUYbf4U,True,38 states have officially approved the boundar...
8,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,GsX_SUh-p7U,True,with all of the news about the pandemic and th...
9,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,sBJnbpNXRRQ,True,music welcome to the lost debate a show for po...


In [34]:
ny_captions_tagged = get_word_lable(ny_captions_clean,'lexicon.json')
ny_captions_tagged

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:6: SettingWithCopyWarn

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text,words,m_label
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there s no ...,"american law,prison,troops,enemies,life begins...","covid,economic,education,environment,foreign p..."
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your nation...,"homeless,police,violent crime,assault,small bu...","covid,economic,safety,social and cultural,soci..."
2,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,music welcome to the lost debate a show for po...,"vaccine mandates,laws,foreign,impeach,consumer...","covid,economic,education,environment,foreign p..."
3,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,UcQHDSvtl90,True,music my dad started at a steel plant and taug...,no words,no topic
4,https://transparencyreport.google.com/politica...,MARKETFUEL SUBSCRIPTION SERVICES,≤ 10k,≤ 100,jG6NGdLUHeM,True,general michael flynn was unmasked at least 48...,"barr,foreign,nsa","foreign policy,governance,safety"
5,https://transparencyreport.google.com/politica...,Friends for Kathy Hochul,≤ 10k,≤ 100,GwiWAmH9Pd4,True,i m fighting to help families keep up with ris...,no words,no topic
6,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah her...,"drugs,guatemala,secure the border,corruption,t...","economic,foreign policy,governance,immigration..."
7,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,zyu4BUYbf4U,True,38 states have officially approved the boundar...,"justice roberts,workers,discrimination,disenfr...","covid,economic,education,governance,immigratio..."
8,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,GsX_SUh-p7U,True,with all of the news about the pandemic and th...,"supreme court,nominated,pandemic,ukraine,black...","covid,foreign policy,governance,safety,social ..."
9,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,sBJnbpNXRRQ,True,music welcome to the lost debate a show for po...,"laws,climate change,discrimination,kavanaugh,p...","covid,economic,education,environment,foreign p..."


In [35]:
count_topic(ny_captions_tagged)

,single_topic,counts
0,economic,25
1,governance,22
2,safety,20
3,social programs,17
4,foreign policy,16
5,social and cultural,15
6,covid,14
7,education,14
8,environment,14
9,health,12


In [36]:
count_word(ny_captions_tagged)

,single_word,counts
0,law,14
1,tax,14
2,pandemic,12
3,school,11
4,education,11
...,...,...
400,crippled,1
401,liberal justice,1
402,life begins,1
403,lithuania,1


# CT

In [37]:
ct_video = get_youtube_id('GoogleAds/CT.csv')
ct_video_clean = check_video(ct_video)
ct_captions = get_captions(ct_video_clean)
ct_captions

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,youtube_id not available: this ad violated goo...,False,\nCould not retrieve a transcript for the vide...
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,Hczz2OL2oZE,True,foreign please rise for our national anthem o...
2,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there's no...
3,https://transparencyreport.google.com/politica...,NRSC,≤ 10k,≤ 100,5beJUEo67Q8,True,\nCould not retrieve a transcript for the vide...
4,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,sBJnbpNXRRQ,True,[Music] welcome to the lost debate a show for...
5,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,[Music] welcome to the lost debate a show for...
6,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,y-trUmtEIsM,True,[Music] welcome to the lost debate a show for...
7,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah he...
8,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your natio...
9,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders sum...


In [40]:
# remove unavialable videos
ct_captions_available = ct_captions[ct_captions['video_available'] == True]
ct_captions_clean = clean_text(ct_captions_available)
ct_captions_clean

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/197128158.py:12: SettingWithCopyWarning: 
A value is try

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,Hczz2OL2oZE,True,foreign please rise for our national anthem oh...
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there s no ...
2,https://transparencyreport.google.com/politica...,NRSC,≤ 10k,≤ 100,5beJUEo67Q8,True,could not retrieve a transcript for the video ...
3,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,sBJnbpNXRRQ,True,music welcome to the lost debate a show for po...
4,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,music welcome to the lost debate a show for po...
5,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,y-trUmtEIsM,True,music welcome to the lost debate a show for po...
6,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah her...
7,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your nation...
8,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...
9,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,ur-kdu386Vw,True,music to the capitol now energy and the enviro...


In [41]:
ct_captions_tagged = get_word_lable(ct_captions_clean,'lexicon.json')
ct_captions_tagged

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_85381/3945606449.py:6: SettingWithCopyWarn

,ad_url,advertiser_name,impressions,spend_usd,youtube_id,video_available,text,words,m_label
0,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,Hczz2OL2oZE,True,foreign please rise for our national anthem oh...,"black lives matter,laws,marijuana,ukraine,tax,...","covid,donation,economic,education,environment,..."
1,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,3tbi7bkvn9g,True,these people actually hate america there s no ...,"american law,prison,troops,enemies,life begins...","covid,economic,education,environment,foreign p..."
2,https://transparencyreport.google.com/politica...,NRSC,≤ 10k,≤ 100,5beJUEo67Q8,True,could not retrieve a transcript for the video ...,no words,no topic
3,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,sBJnbpNXRRQ,True,music welcome to the lost debate a show for po...,"laws,climate change,discrimination,kavanaugh,p...","covid,economic,education,environment,foreign p..."
4,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,UfuAU9dJmvA,True,music welcome to the lost debate a show for po...,"vaccine mandates,laws,foreign,impeach,consumer...","covid,economic,education,environment,foreign p..."
5,https://transparencyreport.google.com/politica...,Lost Debate Inc,≤ 10k,≤ 100,y-trUmtEIsM,True,music welcome to the lost debate a show for po...,"donate,laws,foreign,dissidents,prison,guns,eco...","covid,donation,economic,education,foreign poli..."
6,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,y68knSZLsdA,True,oh there we go hey there we go sam oh yeah her...,"drugs,guatemala,secure the border,corruption,t...","economic,foreign policy,governance,immigration..."
7,https://transparencyreport.google.com/politica...,"TURNING POINT USA, NFP",≤ 10k,≤ 100,2Xn9wTIo-AA,True,what is the state of washington dc your nation...,"homeless,police,violent crime,assault,small bu...","covid,economic,safety,social and cultural,soci..."
8,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,pzLwTOpn9mc,True,morgan president biden opened the leaders summ...,"greenhouse gas,workers,climate,pollution,trans...","economic,environment,health"
9,https://transparencyreport.google.com/politica...,SAHAK NALBANDYAN,≤ 10k,≤ 100,ur-kdu386Vw,True,music to the capitol now energy and the enviro...,"renewables,natural gas,climate change,london,e...","economic,environment,foreign policy"


In [42]:
count_topic(ct_captions_tagged)

,single_topic,counts
0,economic,18
1,foreign policy,16
2,governance,16
3,safety,15
4,covid,14
5,environment,13
6,social and cultural,13
7,social programs,13
8,education,10
9,health,10


In [43]:
count_word(ct_captions_tagged)

,single_word,counts
0,pandemic,12
1,black,10
2,tax,10
3,law,10
4,school,9
...,...,...
390,impeachment,1
391,income scale,1
392,carbon emissions,1
393,inspector general,1
